In [1]:
import os
from PIL import Image
import numpy as np
from random import shuffle
import random

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, GlobalAveragePooling2D, UpSampling2D
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers.core import Lambda
from keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
import skimage as sk
from skimage import util
from skimage.transform import rotate, AffineTransform, warp
import skimage.io as io

from keras.applications.vgg16 import VGG16
from keras import backend as K
import tensorflow as tf
from keras.models import load_model
from keras.models import model_from_json

### Functions to load/prepare data, data augmentation

In [ ]:
def load_alldatas(DIR, IMG_SIZE, aug=2, piv=0):
    ''' 
        Select and load imgs
        resize to IMG_SIZE data. 
        aug: do data augmentation
        piv: amount of images flips to diversify (0 no flip - 4 always flip), useful if no augmentation
        input data directory DIR must have subfolders "input" and "mask" with the raw images and binary files, with same names
    '''
    train_data = []
    cdir = DIR+'/input'
    mdir = DIR+'/mask'
    for img in os.listdir(cdir):
        ## open input image
        path = os.path.join(cdir,img)
        cimg = Image.open(path)
        cimg = cimg.convert('L')
        cimg = cimg.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        cimg = np.array(cimg)
        
        ## open corresponding mask file
        mpath = os.path.join(mdir,img)
        mimg = Image.open(mpath)
        mimg = mimg.convert('L')
        mimg = mimg.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        mimg = np.array(mimg)
        
        ## empty image
        if np.min(cimg)==np.max(cimg):
            print("Something wrong in this image, all empty")
            print(path)
        
        ## applies some image flips
        if random.randrange(4) < piv:
            cimg = np.fliplr(cimg)
            mimg = np.fliplr(mimg)
        if random.randrange(4) < piv:
            cimg = np.flipud(cimg)
            mimg = np.flipud(mimg)
       
        train_data.append( [cimg, mimg, img] )
        
        ## Do basic data augmentation (flips, rotations)
        if aug >= 1:
            shift_img = np.array(cimg)
            shift_img = np.fliplr(shift_img)
            shift_img = np.flipud(shift_img)
            shift_mask = np.array(mimg)
            shift_mask = np.fliplr(shift_mask)
            shift_mask = np.flipud(shift_mask)
            train_data.append( [shift_img, shift_mask, img+'aug.png'] )
            
        if aug >= 2:
            shift_img = np.array(cimg)
            shift_img = np.rot90(shift_img)
            shift_mask = np.array(mimg)
            shift_mask = np.rot90(shift_mask)
            train_data.append( [shift_img, shift_mask, img+'aug2.png'] )
    
    ## randomize the dataset    
    shuffle(train_data)
    shuffle(train_data)
    return train_data

def normalise(img):
    """
    Min-max normalisation
    """
    img = (img - img.min() )/ (img.max()-img.min())
    return img

#### Metrics to train and measure network performance

In [2]:
def mean_iou(y_true, y_pred):
    """
    Score prediction by measuring IOU (Intersection Over Union)
    """
    y_pred_class = K.cast(K.greater(y_pred, .5), dtype='float32') # .5 is the threshold
    tp = tf.reduce_sum(y_pred_class * y_true)
    fp = tf.reduce_sum(tf.nn.relu(y_pred_class-y_true))
    fn = tf.reduce_sum(tf.nn.relu(y_true-y_pred_class))
    
    return tp / (tp + fp + fn)

def jaccard_distance(y_true, y_pred, smooth=100):
    """Jaccard distance for semantic segmentation.
    Also known as the intersection-over-union loss.
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    Used to calculate loss during training
    """
    #y_pred = K.cast(K.greater(y_pred, .5), dtype='float32')
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ =tf.reduce_sum(y_true+y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1-jac)*smooth


#### Network implementation, training, save/load network

In [3]:
def load_model(nameMod):
    """
    Load the weights of a trained network from file
    """
    json_file = open(nameMod+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(nameMod+".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[mean_iou, 'accuracy'] )
    return loaded_model

def save_model(model, nameMod):
    """
    Save trained model (network) to files, tensorflow format
    """
    # serialize model to JSON
    model_json = model.to_json()
    with open(nameMod+".json", "w") as json_file:
        json_file.write(model_json)
        # serialize weights to HDF5
        model.save_weights(nameMod+".h5")
        print("Saved model to disk")

def run_model( train_imgs, train_labels, batch_size=10, epochs=2, IMG_SIZE=224):
    """ Define and initialize the network and do the training """
    model = build_unet(IMG_SIZE)
    model.fit( train_imgs, train_labels, batch_size=batch_size, epochs=epochs, verbose=1)
    return model

def train_model(train_data, IMG_SIZE, batchy=1, epoch=4):
    """
    Go, do the training: preprocess images, and run training
    """
    train_images = np.array([normalise(i[0]) for i in train_data]).reshape(-1,IMG_SIZE, IMG_SIZE, 1)
    train_mask = np.array([i[1] for i in train_data]).reshape(-1,IMG_SIZE, IMG_SIZE, 1)
    train_mask = (train_mask/255)
    train_names = np.array([i[2] for i in train_data])
    
    ## Run network and save
    model = run_model( train_imgs=train_images, train_labels=train_mask, batch_size=batchy, epochs=epoch, IMG_SIZE=IMG_SIZE)
    return [model, train_mean, train_std]
    

def conv_block( nfils, inputs):
    """
    Convolution block in U-Net network: sequence of 2 convolutions and normalisation
    """
    conv = Conv2D(nfils, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv = BatchNormalization()(conv)
    conv = Conv2D(nfils, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv)
    conv = BatchNormalization()(conv)
    return conv


def build_unet(IMG_SIZE):
    """
    U-Net architecture, with number of initial features choosable (nfil)
    """
    inputs = Input((IMG_SIZE, IMG_SIZE, 1))
    nfil = 8   ## cortex model
    #nfil = 24  ## zp model
     
    ## Downward part of U-Net
    conv1 =  conv_block( nfil, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 =  conv_block( nfil*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 =  conv_block( nfil*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 =  conv_block( nfil*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 =  conv_block( nfil*16, pool4)
    
    ## Upward part of U-Net with skip connections
    up6 = Conv2D(nfil*8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = conv_block( nfil*8, merge6 )
    
    up7 = Conv2D(nfil*4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = conv_block( nfil*4, merge7 )

    up8 = Conv2D(nfil*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = conv_block( nfil*2, merge8 )
    
    up9 = Conv2D(nfil*1, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = conv_block( nfil*1, merge9 )
    
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs = inputs, outputs = conv10)

    ### load a pretrained model, and freeze the first layers
    pretrained_model = load_model("/home/gaelle/Ext/MIV/humanMovies/trainingOocytor/models/cortex/model0/resmodel0")
    ##pretrained_model = load_model("/home/gaelle/Ext/MIV/humanMovies/trainingOocytor/models/zp/run0/resmodel0") # zp model
    for ind in range(len(model.layers)):
        cind = ind
        layer = model.layers[cind]
        pretrained_layer = pretrained_model.layers[cind]
        layer.set_weights(pretrained_layer.get_weights())
        if cind<3:
            layer.trainable = False
    
    model.compile(optimizer = 'adam', loss = jaccard_distance, metrics = [mean_iou, 'accuracy'] )
    return model


# Main - Load the training data, and retrain the network

In [ ]:
## parameters
path = "/home/gaelle/Ext/MIV/humanMovies/"
TRAINDIR = path+"dataGT/cortex/"     ## directory of training data (with subfolders "input" and "mask")
outfold = path+"trainingOocytor/models/cortex/retrained/retrained4b/"    ## where the retrained network will be saved
IMG_SIZE = 256   ## size of the images in Oocytor
nepoch = 40      ## number of retraining iterations

## load data
accur = []
train_data = []
print("Load training data")
train_oursinfert = load_alldatas(TRAINDIR, IMG_SIZE, 2, 3)
for cur in train_oursinfert:
    train_data.append(cur)    
    shuffle(train_data)
shuffle(train_data)

## use GPU if possible
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
config = tf.compat.v1.ConfigProto(
      #device_count = {'GPU': 0}
    )
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

## train model
model, train_mean, train_std = train_model(train_data, IMG_SIZE, batchy=30, epoch=nepoch)
save_model(model, outfold+"resmodel"+str(count)+"")

Load training data
True
2.8.0
Num GPUs Available:  1


2022-12-19 15:22:19.941714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20911 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6


Loaded model from disk
Epoch 1/40
57/57 [==============================] - 6s 73ms/step - loss: 3.4874 - mean_iou: 0.9636 - accuracy: 0.9720
Epoch 2/40
57/57 [==============================] - 4s 74ms/step - loss: 2.9604 - mean_iou: 0.9694 - accuracy: 0.9733
Epoch 3/40
57/57 [==============================] - 4s 74ms/step - loss: 2.6720 - mean_iou: 0.9726 - accuracy: 0.9741
Epoch 4/40
57/57 [==============================] - 4s 74ms/step - loss: 2.4707 - mean_iou: 0.9749 - accuracy: 0.9746
Epoch 5/40
57/57 [==============================] - 4s 73ms/step - loss: 2.3828 - mean_iou: 0.9758 - accuracy: 0.9748
Epoch 6/40
57/57 [==============================] - 4s 73ms/step - loss: 2.2495 - mean_iou: 0.9772 - accuracy: 0.9750
Epoch 7/40
57/57 [==============================] - 4s 72ms/step - loss: 2.1732 - mean_iou: 0.9780 - accuracy: 0.9752
Epoch 8/40
57/57 [==============================] - 4s 75ms/step - loss: 2.1224 - mean_iou: 0.9785 - accuracy: 0.9753
Epoch 9/40
57/57 [===============

#### test performance on test dataset

In [6]:

## test fertilized
test_data = load_alldatas(TESTDIR, IMG_SIZE, 0)
test_images = np.array([normalise(i[0]) for i in test_data]).reshape(-1,IMG_SIZE, IMG_SIZE,1)
test_labels = np.array([i[1] for i in test_data]).reshape(-1,IMG_SIZE, IMG_SIZE,1)
test_names = np.array([i[2] for i in test_data])
test_labels = test_labels/255
loss, iou, acc = model.evaluate( test_images, test_labels, verbose=1)
print(str(loss)+' '+str(iou)+' '+str(acc))
resfile = open(outfold+"test_retrained.txt", "w")
resfile.write(str(loss)+' '+str(iou)+' '+str(acc))
resfile.close()

NameError: name 'TESTDIR' is not defined